In [1]:
import numpy as np
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os
from ProjectF import MLAData,classification, Object,storing,MLADataBin
import random

In [2]:

TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")

In [3]:
PlateDir ="D:"+slash+"Data"+slash+"File.txt"
with open(PlateDir) as f:
    Spectra_Files = f.read().splitlines()

0 = flux
1 = class
2 = or
3 = and
4 = inv
5 = red

In [4]:
wav_log=[]
ORMASK=[]
ANDMASK=[]
INVAR=[]
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir+slash+ spectrum ,memmap=True)
    primhdu_ = plate_[0]
    hdu =  plate_[1].data
    wav_log.append(primhdu_.header['Plate'])
    ORMASK.append( hdu[2])
    ANDMASK.append( hdu[3])
    Flux.append(plate_[1].data)

In [11]:
p=0

while

In [18]:
b.info()

Filename: D:\Data\Training\spPlate-7374-56751.fits
No.    Name         Type      Cards   Dimensions   Format
  0  PRIMARY     PrimaryHDU     231   (4647, 1000)   float32   
  1  IVAR        ImageHDU        16   (4647, 1000)   float32   
  2  ANDMASK     ImageHDU        15   (4647, 1000)   int32   
  3  ORMASK      ImageHDU        15   (4647, 1000)   int32   
  4  WAVEDISP    ImageHDU        16   (4647, 1000)   float32   
  5  PLUGMAP     BinTableHDU     90   1000R x 36C   [5J, 6A, D, D, 5E, E, E, E, 16A, D, D, J, J, J, J, J, J, E, 15A, E, E, J, K, K, K, K, J, 5A, J, J, J, K, 5E, 5E, 5J, E]   
  6  SKY         ImageHDU         8   (4647, 1000)   float32   


In [19]:
b[2].data

array([[16777217, 16777217, 16777217, ..., 16777217, 16777217, 16777217],
       [       0,        0,        0, ..., 83886080, 83886080, 83886080],
       [       4,        4,        4, ..., 83886084, 83886084, 83886084],
       ..., 
       [83886080, 83886080, 83886080, ..., 16777216, 16777216, 83886080],
       [83886080, 83886080, 83886080, ..., 16777216, 16777216, 83886080],
       [83886080, 83886080, 83886080, ..., 16777216, 16777216, 83886080]])

In [ ]:
a= plate_[1].data

In [ ]:
b=a[0]
b